In [86]:
# Importamos librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest


In [87]:
#cargamos el dataset de estudio 
dataset= pd.read_csv('AA_Sample_Particulars_District.csv', sep=',', header=0)
dataset

,State_Name,State_District_Name,AA_Sample_Units_Total,AA_Sample_Units_Rural,AA_Sample_Units_Urban,AA_Households_Total,AA_Households_Rural,AA_Households_Urban,AA_Population_Total,AA_Population_Rural,AA_Population_Urban,AA_Ever_Married_Women_Aged_15_49_Years_Total,AA_Ever_Married_Women_Aged_15_49_Years_Rural,AA_Ever_Married_Women_Aged_15_49_Years_Urban,AA_Currently_Married_Women_Aged_15_49_Years_Total,AA_Currently_Married_Women_Aged_15_49_Years_Rural,AA_Currently_Married_Women_Aged_15_49_Years_Urban,AA_Children_12_23_Months_Total,AA_Children_12_23_Months_Rural,AA_Children_12_23_Months_Urban
0,Assam,Barpeta,53,47,6,13711,12765,946,64606,60293,4313,12541.0,11692.0,849,11876.0,11075.0,801,2371,2261,110
1,Assam,Bongaigaon,89,73,16,17384,14904,2480,84012,73899,10113,16781.0,14591.0,2190,15533.0,13566.0,1967,2451,2259,192
2,Assam,Cachar,105,84,21,27488,24207,3281,127158,112171,14987,23693.0,20881.0,2812,21536.0,18957.0,2579,5709,5122,587
3,Assam,Darrang,26,24,2,5951,5769,182,29048,28256,792,5424.0,5277.0,147,5027.0,4886.0,141,984,950,34
4,Assam,Dhemaji,121,108,13,14481,12619,1862,75451,66537,8914,14159.0,12392.0,1767,13529.0,11856.0,1673,3467,3103,364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,Uttarakhand,Pithoragarh,161,136,25,25265,20467,4798,102999,84627,18372,21408.0,17463.0,3945,20015.0,16283.0,3732,3358,2861,497
280,Uttarakhand,Rudraprayag,225,222,3,33117,32914,203,139025,138266,759,28470.0,28330.0,140,26788.0,26656.0,132,5090,5070,20
281,Uttarakhand,Tehri Garhwal,351,317,34,47792,42354,5438,205575,181918,23657,40347.0,35894.0,4453,38198.0,33893.0,4305,6946,6306,640
282,Uttarakhand,Udham Singh Nagar,81,45,36,17887,12393,5494,91518,64614,26904,16133.0,11268.0,4865,15414.0,10757.0,4657,3188,2234,954


In [88]:
dataset[dataset['State_District_Name']== 'Barpeta']

,State_Name,State_District_Name,AA_Sample_Units_Total,AA_Sample_Units_Rural,AA_Sample_Units_Urban,AA_Households_Total,AA_Households_Rural,AA_Households_Urban,AA_Population_Total,AA_Population_Rural,AA_Population_Urban,AA_Ever_Married_Women_Aged_15_49_Years_Total,AA_Ever_Married_Women_Aged_15_49_Years_Rural,AA_Ever_Married_Women_Aged_15_49_Years_Urban,AA_Currently_Married_Women_Aged_15_49_Years_Total,AA_Currently_Married_Women_Aged_15_49_Years_Rural,AA_Currently_Married_Women_Aged_15_49_Years_Urban,AA_Children_12_23_Months_Total,AA_Children_12_23_Months_Rural,AA_Children_12_23_Months_Urban
0,Assam,Barpeta,53,47,6,13711,12765,946,64606,60293,4313,12541.0,11692.0,849,11876.0,11075.0,801,2371,2261,110


In [89]:
nobs_global= np.array([dataset['AA_Population_Total'].sum(),dataset['AA_Sample_Units_Total'].sum()])
count_global= np.array([dataset['AA_Population_Rural'].sum(), dataset['AA_Sample_Units_Rural'].sum()])
stat, pval = proportions_ztest(count=count_global, nobs=nobs_global)
print(stat, pval)
print('Comparacion de proporciones con valor p menor a cualquier significacion posible. Esto indica que la proporcion de personas rurales del total de la poblacion', 
'y la proporcion de personas rurales de la muestra total, no son estadisticamete iguales.')

27.854315160936512 9.550087807884865e-171
Comparacion de proporciones con valor p menor a cualquier significacion posible. Esto indica que la proporcion de personas rurales del total de la poblacion y la proporcion de personas rurales de la muestra total, no son estadisticamete iguales.


In [90]:
# Creamos el data frame en la que incluiremos las proporciones por distrito 

contingency_table = pd.DataFrame({ 'State_District_Name': dataset['State_District_Name'],
     'Total_Poblacion': dataset['AA_Population_Total'],
    'Poblacion_Rural': dataset['AA_Population_Rural'],
    'Total_Muestra': dataset['AA_Sample_Units_Total'],
    'Muestra_Rural': dataset['AA_Sample_Units_Rural']
})
# Calculamos las proporciones y las incluimos al dataframe anterior
contingency_table['p_poblacional'] = contingency_table['Poblacion_Rural'] / contingency_table['Total_Poblacion']
contingency_table['p_muestral'] = contingency_table['Muestra_Rural'] / contingency_table['Total_Muestra']

# Aplicar la prueba de proporciones individualmente por distrito
p_values = []
for index in range(len(contingency_table)):
    count = np.array([contingency_table['Poblacion_Rural'][index], contingency_table['Muestra_Rural'][index]])
    nobs = np.array([contingency_table['Total_Poblacion'][index], contingency_table['Total_Muestra'][index]])
    
    # Solo ejecutar si ambas poblaciones son mayores a 0
    if all(nobs > 0):
        stat, pval = proportions_ztest(count=count, nobs=nobs)
    else:
        pval = np.nan  # Si hay ceros en el denominador, marcamos como NaN
        
    p_values.append(pval)

# Agregar resultados a la tabla. Calculamos el valor p correspondiente al estadistico z calculado en esta prueba de comparación de proporciones. 
# La significancia elegida es del 10%. Quiere decir que si no rechamos Ho, estamos concluyendo con un 90% de confianza que las proporciones son iguales.
contingency_table['p-valor'] = p_values
contingency_table['Muestra_Representativa'] = contingency_table['p-valor'].apply(lambda x: "Sí" if x > 0.10 else "No")

contingency_table


,State_District_Name,Total_Poblacion,Poblacion_Rural,Total_Muestra,Muestra_Rural,p_poblacional,p_muestral,p-valor,Muestra_Representativa
0,Barpeta,64606,60293,53,47,0.933241,0.886792,0.175783,Sí
1,Bongaigaon,84012,73899,89,73,0.879624,0.820225,0.085285,No
2,Cachar,127158,112171,105,84,0.882139,0.800000,0.009092,No
3,Darrang,29048,28256,26,24,0.972735,0.923077,0.120454,Sí
4,Dhemaji,75451,66537,121,108,0.881857,0.892562,0.715451,Sí
...,...,...,...,...,...,...,...,...,...
279,Pithoragarh,102999,84627,161,136,0.821629,0.844720,0.444384,Sí
280,Rudraprayag,139025,138266,225,222,0.994541,0.986667,0.109663,Sí
281,Tehri Garhwal,205575,181918,351,317,0.884923,0.903134,0.285352,Sí
282,Udham Singh Nagar,91518,64614,81,45,0.706025,0.555556,0.002970,No


In [91]:
# Creamos el data frame en la que incluiremos las proporciones por estado 

contingency_table_2 = pd.DataFrame({
     'Total_Poblacion': dataset.groupby(['State_Name'])['AA_Population_Total'].sum(),
    'Poblacion_Rural': dataset.groupby(['State_Name'])['AA_Population_Rural'].sum(),
    'Total_Muestra': dataset.groupby(['State_Name'])['AA_Sample_Units_Total'].sum(),
    'Muestra_Rural': dataset.groupby(['State_Name'])['AA_Sample_Units_Rural'].sum()
})
# Calculamos las proporciones y las incluimos al dataframe anterior
contingency_table_2['p_poblacional'] = contingency_table_2['Poblacion_Rural'] / contingency_table_2['Total_Poblacion']
contingency_table_2['p_muestral'] = contingency_table_2['Muestra_Rural'] / contingency_table_2['Total_Muestra']

# Aplicar la prueba de proporciones individualmente por distrito
p_values = []
for index in range(len(contingency_table_2)):
    count = np.array([contingency_table_2.iloc[index]['Poblacion_Rural'], contingency_table_2.iloc[index]['Muestra_Rural']])
    nobs = np.array([contingency_table_2.iloc[index]['Total_Poblacion'], contingency_table_2.iloc[index]['Total_Muestra']])
    
    # Solo ejecutar si ambas poblaciones son mayores a 0
    if all(nobs > 0):
        stat, pval = proportions_ztest(count=count, nobs=nobs)
    else:
        pval = np.nan  # Si hay ceros en el denominador, marcamos como NaN
        
    p_values.append(pval)

# Agregar resultados a la tabla. Calculamos el valor p correspondiente al estadistico z calculado en esta prueba de comparación de proporciones. 
# La significancia elegida es del 10%. Quiere decir que si no rechamos Ho, estamos concluyendo con un 90% de confianza que las proporciones son iguales.
contingency_table_2['p-valor'] = p_values
contingency_table_2['Muestra_Representativa'] = contingency_table_2['p-valor'].apply(lambda x: "Sí" if x > 0.1 else "No")

contingency_table_2

,Total_Poblacion,Poblacion_Rural,Total_Muestra,Muestra_Rural,p_poblacional,p_muestral,p-valor,Muestra_Representativa
State_Name,,,,,,,,
Assam,1809610,1518639,1784,1412,0.839208,0.791480,4.145847e-08,No
Bihar,3227867,2993906,2356,1981,0.927518,0.840832,3.917641e-59,No
Chhattisgarh,1264309,994416,1255,926,0.786529,0.737849,2.595853e-05,No
Jharkhand,2019298,1644036,2108,1513,0.814162,0.717742,5.718958e-30,No
Madhya Pradesh,2389787,1629355,2557,1660,0.681799,0.649198,4.042360e-04,No
Odisha,1992799,1637152,2364,1798,0.821534,0.760575,1.036659e-14,No
Rajasthan,1828116,1521017,1841,1294,0.832013,0.702879,1.288807e-49,No
Uttar Pradesh,4808503,3883293,3927,2782,0.807589,0.708429,6.378286e-56,No
Uttarakhand,1726477,1365247,2501,1962,0.790770,0.784486,4.400777e-01,Sí


In [47]:
# Exportamos los data frames por distrito y por estado:

contingency_table.to_csv("contingency_table.csv", index=False)
contingency_table_2.to_csv("contingency_table_2.csv", index=True)